In [1]:

#print hdu[0].header.cards

import Analysis, Tools
import SourceMap
from astropy.wcs import WCS
from scipy.sparse import csr_matrix

from astropy import units as u
from astropy.coordinates import SkyCoord
reload(Analysis)

def AddExtendedSource(self, source, template_dir='/data/Extended_archive_v15/Templates/', fixed=True):
    """
    Add a 3FGL extended source to the analysis. 
    :param source: the FGL source name 
    """

    # Open FGL and look up the source name.
    hdu = pyfits.open(self.fglpath)
#     print hdu[5].data['2FGL_Name']
#     print hdu[5].header.cards
    # Find what the 3FGL catalog source name is of the requested source
    idx = -1
    for i_ext, name in enumerate(hdu[5].data['Source_Name']):
        if name == source:
            idx = i_ext
            break
    if idx == -1: 
        raise Exception('Extended source not %s found!'%source)
    #print 'source found at idx %i'%idx
    # Find the index of the extended source in the main catalog.
    idx_fgl = np.where(hdu[1].data['Source_Name']==hdu[5].data['2FGL_Name'][idx])[0]
    # Read the spatial template filename
    fname = hdu[5].data['Spatial_Filename'][idx]
    # Open the file and remove erroneous header info. 
    hdu_spatial = pyfits.open(template_dir+'/'+fname, 'readonly')
    try:
        hdu_spatial[0].header.pop('COMMENT')
    except:
        pass
    try:
        hdu_spatial[0].header.pop('HISTORY')
    except:
        pass
    
    # Read the WCS coordinates
    w = WCS(hdu_spatial[0].header, relax=False, fix=True)
    
    
    # Init a blank healpix template
    t = np.zeros(12*self.nside**2, dtype=np.float32)
    # Map the FGL extended template into healpix space, row by row. 
    for i_row in range(hdu_spatial[0].data.shape[0]):
        # get lat/lon for each row
        lon, lat = w.all_pix2world(i_row,np.arange(hdu_spatial[0].data.shape[1]), 0)
        c_icrs = SkyCoord(ra=lon, dec=lat, frame='icrs', unit='degree')
        lon,lat = c_icrs.galactic.l.degree, c_icrs.galactic.b.degree
        # transform lat/lon to healpix
        hpix_idx = Tools.ang2hpix(lon, lat, nside=self.nside)
        # Add these counts to the healpix template
        np.add.at(t, hpix_idx, hdu_spatial[0].data[i_row,:])
        
        
    # Get the total number of counts from this source in each energy bin.  This will set the normalization
    total_counts = np.sum(self.GenPointSourceTemplate(pscmap=None, onlyidx=[idx_fgl,], save=False, verbosity=0), axis=1)
    print total_counts
    
    # Generate a master sourcemap for the extended source (spatial template for each energy bin). 
    master_t = np.array([t for i_E, count in enumerate(total_counts)])
    for i_E, count in enumerate(total_counts):
        #master_t[i_E] = master_t[i_E]/np.sum(master_t[i_E])*count # normed to the expected PSC flux
        # Apply the PSF 
        master_t[i_E] = Tools.ApplyGaussianPSF(master_t[i_E], E_min=self.bin_edges[i_E], E_max=self.bin_edges[i_E+1], psfFile=self.psfFile, multiplier=1.)
    
    # Convert to sparse matrix for memory profile. 
    t_sparse = csr_matrix(master_t , dtype=np.float32)
    self.AddTemplate(source.replace(' ',''), t_sparse, fixSpectrum=fixed, fixNorm=fixed,
                             limits=[None, None], value=1., ApplyIRF=False, sourceClass='FGL')
    

    
    
    
A = Analysis.Analysis()

A.PopulateROI([0,0],radius=360, fix_radius=360, include_point=False)

import healpy
#healpy.cartview(A.templateList['CenALobes'].healpixCube[0].toarray()[0])

Populating ROI with point sources: 123 of 3034
Populating ROI with point sources: 603 of 3034
Populating ROI with point sources: 640 of 3034
Populating ROI with point sources: 697 of 3034

ERROR: KeyboardInterrupt [numpy.core._methods]
ERROR:astropy:KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
hdu = pyfits.open('/data/Extended_archive_v15/Templates//S147.fits')
hdu.info()

Filename: /data/Extended_archive_v15/Templates//S147.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      22   (95, 95)     float32   


In [68]:
A.AddTemplate(name='PSCExt', healpixCube=master, fixSpectrum=True, fixNorm=True,ApplyIRF=False, noPSF=True)

                NAME                    LIMITS      VALUE    FIXNORM    FIXSPEC   SRCCLASS
           CenALobes              [None, None]  1.000e+00       True       True        FGL


In [123]:
c_icrs = SkyCoord(ra=2*u.degree, dec=1*u.degree, frame='icrs')

In [19]:
A.PrintTemplates()

                NAME                    LIMITS      VALUE    FIXNORM    FIXSPEC   SRCCLASS
       HESSJ1841-055              [None, None]  1.000e+00      False      False        FGL
             PuppisA              [None, None]  1.000e+00      False      False        FGL
           CenALobes              [None, None]  1.000e+00      False      False        FGL
            MSH15-52              [None, None]  1.000e+00      False      False        FGL
       HESSJ1303-631              [None, None]  1.000e+00      False      False        FGL
       HESSJ1614-518              [None, None]  1.000e+00      False      False        FGL
                 SMC              [None, None]  1.000e+00      False      False        FGL
                 W28              [None, None]  1.000e+00      False      False        FGL
                 W44              [None, None]  1.000e+00      False      False        FGL
              VelaJr              [None, None]  1.000e+00      False      False        FGL